형태소 분석기 비교 : https://soohee410.github.io/compare_tagger  
Song2Vec : https://velog.io/@mstar228/Song2Vec

## 감성분석
- 맛집 리뷰를 바탕으로 감성분석하기

### 0.라이브러리 임포트

In [66]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
# 지표를 하나만 설정할 경우
from sklearn.model_selection import cross_val_score
# 지표를 하나 이상 설정할 경우
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 모델의 최적의 하이퍼파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 군집화
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth



# ARIMA (시계열 예측)
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm


# 시간 측정을 위한 시간 모듈
import datetime

# 주식정보
from pandas_datareader import data

# 형태소 벡터를 생성하기 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
# 형태소 벡터를 학습 벡터로 변환한다.
from sklearn.feature_extraction.text import TfidfTransformer


# 데이터 수집
import requests
from bs4 import BeautifulSoup
import re
import time
import os
import json

# 한국어 형태소 분석
from konlpy.tag import Okt, Hannanum, Kkma, Mecab, Komoran

# 워드 클라우드를 위한 라이브러리
from collections import Counter
import pytagcloud
from IPython.display import Image

# 저장
import pickle

### 1.데이터 로드

In [16]:
df = pd.read_csv("221204_review_data.csv")
df.head(2)

,score,review,y
0,5,친절하시고 깔끔하고 좋았습니다,1
1,5,조용하고 고기도 굿,1


### 2.데이터전처리

In [28]:
def text_clearing(text): 
	hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    # 지정한 정규식에 해당하지 않은 것은 길이가 0인 문자열로 변환한다.
	result = hangul.sub('', text)
	return result

In [29]:
# 리뷰에 함수 적용하여 ko_review 컬럼을 생성하기
df["ko_review"] = df["review"].apply(lambda x : text_clearing(x))

In [30]:
# 리뷰 컬럼 삭제
df.drop("review", axis=1, inplace=True)
df.head(2)

,score,y,ko_review
0,5,1,친절하시고 깔끔하고 좋았습니다
1,5,1,조용하고 고기도 굿


### 3.형태소분석

In [31]:
# konlpy 라이브러리로 텍스트 데이터에서 형태소를 추출한다.
def get_pos (x) :
    tagger = Okt()
    pos = tagger.pos(x)
    
    # 단어와 품사를 합쳐서 하나의 단어로 만들어준다.'동/Modifier'
    result = []
    
    # 형태소의 수만큼 반복한다.
    # 조사인 것과 명사인 것이 같을 수 있기 때문에 구분해준다.
    # 형태소 벡터를 만들때 추후 사용
    for a1 in pos :
        result.append(f'{a1[0]}/{a1[1]}')
    
    return result

In [32]:
get_pos("동해물과 백두산이 마르고 닳도록")

['동/Modifier',
 '해물/Noun',
 '과/Josa',
 '백두산/Noun',
 '이/Josa',
 '마르고/Noun',
 '닳도록/Verb']

### 4.형태소 벡터화

In [111]:
index_vectorizer = CountVectorizer(tokenizer=lambda x : get_pos(x))
X = index_vectorizer.fit_transform(df["ko_review"].tolist())
X

<545x3030 sparse matrix of type '<class 'numpy.int64'>'
	with 9692 stored elements in Compressed Sparse Row format>

In [133]:
index_vectorizer.fit(['수익성은 어떻게 되나요'])
print(index_vectorizer.get_feature_names())
print(index_vectorizer.vocabulary_)

['되나요/Verb', '성은/Noun', '수익/Noun', '어떻게/Adjective']
{'수익/Noun': 2, '성은/Noun': 1, '어떻게/Adjective': 3, '되나요/Verb': 0}


In [123]:
print(X)

  (0, 2647)	1
  (0, 428)	1
  (0, 2403)	1
  (1, 2356)	1
  (1, 233)	1
  (1, 721)	1
  (1, 330)	1
  (2, 120)	1
  (2, 260)	1
  (2, 528)	1
  (2, 2065)	1
  (2, 1419)	1
  (2, 2082)	1
  (2, 1013)	1
  (3, 2082)	1
  (3, 671)	1
  (3, 2604)	1
  (3, 2067)	1
  (3, 956)	1
  (3, 1996)	1
  (3, 2077)	1
  (3, 293)	1
  (3, 1476)	1
  (3, 1705)	1
  (4, 233)	1
  :	:
  (543, 721)	1
  (543, 1509)	1
  (543, 2074)	1
  (543, 6)	1
  (543, 2640)	1
  (543, 2934)	1
  (543, 2586)	1
  (544, 721)	1
  (544, 2065)	1
  (544, 1419)	1
  (544, 2069)	1
  (544, 1265)	1
  (544, 2365)	1
  (544, 1447)	1
  (544, 1479)	1
  (544, 14)	1
  (544, 155)	1
  (544, 1348)	1
  (544, 2314)	1
  (544, 543)	1
  (544, 1028)	1
  (544, 1027)	1
  (544, 1371)	1
  (544, 2100)	1
  (544, 2740)	1


In [37]:
index_vectorizer.vocabulary_ # 하나의 단어장으로 저장된 것을 불러옴
# 형태소별로 다른 번호가 부여됨

{'친절하시고/Adjective': 2647,
 '깔끔하고/Adjective': 428,
 '좋았습니다/Adjective': 2403,
 '조용하고/Adjective': 2356,
 '고기/Noun': 233,
 '도/Josa': 721,
 '굿/Noun': 330,
 '갈비탕/Noun': 120,
 '과/Josa': 260,
 '냉면/Noun': 528,
 '육회/Noun': 2065,
 '비빔밥/Noun': 1419,
 '이/Josa': 2082,
 '맛있습니다/Adjective': 1013,
 '대/Modifier': 671,
 '체적/Noun': 2604,
 '으로/Josa': 2067,
 '만족하나/Adjective': 956,
 '와인/Noun': 1996,
 '의/Josa': 2077,
 '구성/Noun': 293,
 '살짝/Noun': 1476,
 '아쉬움/Noun': 1705,
 '맛있고/Adjective': 1001,
 '서비스/Noun': 1508,
 '는/Josa': 589,
 '더/Noun': 701,
 '최고/Noun': 2613,
 '입니다/Adjective': 2182,
 '가/Josa': 24,
 '입/Noun': 2177,
 '에서/Josa': 1897,
 '녹아요/Verb': 553,
 '였습니다/Verb': 1935,
 '살살/Noun': 1473,
 '녹는/Verb': 552,
 '최상급/Noun': 2615,
 '소고기/Noun': 1540,
 '를/Josa': 901,
 '맛/Noun': 995,
 '보고왔습니다/Verb': 1316,
 '구워주고/Verb': 302,
 '가성/Noun': 73,
 '비/Noun': 1408,
 '짱/Noun': 2528,
 '콜키/Noun': 2688,
 '지/Josa': 2477,
 '프리/Noun': 2788,
 '라서/Josa': 885,
 '을/Josa': 2070,
 '가지/Noun': 89,
 '고/Josa': 227,
 '가면/Noun': 61,
 '좋은/Adjective

In [38]:
# 첫번째 행의 리뷰 이름
print(df["ko_review"][0])
# 첫번째 행의 긍정/부정 시그널
print(df["y"][0])
# 첫번째 행의 index_vectorizer의 (형태소, 시리얼넘버)  
print(X[0])

친절하시고 깔끔하고 좋았습니다
1
  (0, 2647)	1
  (0, 428)	1
  (0, 2403)	1


In [42]:
print(X[0])

  (0, 2647)	1
  (0, 428)	1
  (0, 2403)	1


### 5.형태소별 벡터를 TF-IDF 벡터로 생성
- (단어 빈도-역 문서 빈도: Term Frequency-Inverse Document Frequency)

In [ ]:
# 머신 러닝에서 정규화와 비슷한 개념을 수행하는 것임

In [45]:
# TFidf 변환 모델 생성
tfidf_vectorizer = TfidfTransformer()
# 형태소 벡터 변환하기
X = tfidf_vectorizer.fit_transform(X)
print(X[0])

  (0, 2647)	0.5548708693511647
  (0, 2403)	0.48955631270748484
  (0, 428)	0.6726462183300624


### 6.학습 데이터 생성

In [46]:
# 데이터 프레임의 y행을 시리즈로 저장
y = df["y"]

#### 6-1.최적의 학습 데이터를 선정하기 위해서 KNN, 로지스틱 회귀, SVC, 랜덤포레스트, XGBoost의 
#### 최적의 하이퍼 파라미터 구하기

In [47]:
# KNN
params = {
	"n_neighbors" : list(range(1, 11))
}

# 사용할 모델 객체 생성
model1 = KNeighborsClassifier()

# 최적의 하이퍼 파라미터를 찾는다. 
kfold = KFold(n_splits=10, shuffle=True, random_state = 1)
grid_clf1 = GridSearchCV(model1, param_grid = params, scoring="f1", cv=kfold)
grid_clf1.fit(X, y)

# 결과 출력
print(f"최적의 하이퍼 파라미터 : {grid_clf1.best_params_}")
print(f"최적의 모델 평균 성능 : {grid_clf1.best_score_}")

최적의 하이퍼 파라미터 : {'n_neighbors': 4}
최적의 모델 평균 성능 : 0.9561087805947686


In [49]:
# LogisticRegression 
# penalty : 규제의 종류(l1, l2, elasticnet, none)
# C : 규제의 강도 
params = {
    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
}

model2 = LogisticRegression()

grid_clf2 = GridSearchCV(model2, param_grid=params, scoring='f1', cv=kfold)
grid_clf2.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf2.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf2.best_score_}')

최적의 하이퍼 파라미터 : {'C': 0.0001, 'penalty': 'none'}
최적의 모델 평균 성능 : 0.9580627713400792


In [48]:
# SVM(SVC)
# SVM은 penalty가 l2로 고정되어 있다
# C : 규제의 강도 
params = {
    'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
}

model3 = SVC()

grid_clf3 = GridSearchCV(model3, param_grid=params, scoring='f1', cv=kfold)
grid_clf3.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf3.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf3.best_score_}')

최적의 하이퍼 파라미터 : {'C': 0.0001}
최적의 모델 평균 성능 : 0.9479277809113741


In [53]:
# RandomForest
# n_estimators : 사용할 트리의 개수
# max_depth : 생성될 최대 질문 깊이, None은 무한대.
params = {
    'n_estimators' : [50, 100, 150, 200, 250, 300],
    'max_depth' : [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

model4 = RandomForestClassifier(random_state=1)

grid_clf4 = GridSearchCV(model4, param_grid=params, scoring='f1', cv=kfold)
grid_clf4.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf4.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf4.best_score_}')

최적의 하이퍼 파라미터 : {'max_depth': None, 'n_estimators': 50}
최적의 모델 평균 성능 : 0.9479277809113741


In [51]:
# XGBoost
# booster : 내부에 사용할 알고리즘
# learning_rate : 학습률
# n_estimators : 사용할 트리의 개수
# max_depth : 생성될 최대 질문 깊이, None은 무한대.

params = {
    'booster' : ['gbtree', 'gblinear'],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
    'n_estimators' : [50, 100, 150, 200, 250, 300],
    # 'max_depth' : [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

model5 = XGBClassifier(silent=True, verbosity=0)

grid_clf5 = GridSearchCV(model5, param_grid=params, scoring='f1', cv=kfold)
grid_clf5.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf5.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf5.best_score_}')

최적의 하이퍼 파라미터 : {'booster': 'gblinear', 'learning_rate': 0.1, 'n_estimators': 100}
최적의 모델 평균 성능 : 0.9512263550298089


#### 6-2.최고의 모델을 선정

In [54]:
print(grid_clf1.best_score_) # KNN
print(grid_clf2.best_score_) # LogisticRegression 
print(grid_clf3.best_score_) # SVM(SVC)
print(grid_clf4.best_score_) # RandomForest
print(grid_clf5.best_score_) # XGBoost

0.9561087805947686
0.9580627713400792
0.9479277809113741
0.9479277809113741
0.9512263550298089


### 7.중요 단어 파악
- 가장 성능이 좋았던 모델 로지스틱회귀를 선택

In [55]:
model = grid_clf2.best_estimator_

# 상관관계수 구하기
a1 = (model.coef_[0])  # coef_함수를 이용하면 학습 모델에서 사용한 모든 칼럼과 결과 데이터와의 상관계수를 알 수 있음
display(a1)
len(a1) # a1에는 총 3030개의 칼럼이 저장(형태소의 개수 : 3030)

# 칼럼의 개수만큼 나오며, 
# 값이 클수록 1(긍정적인 반응)과 상관관계가 높고, 값이 작을수록 0(부정적인 반응)과 상관관계가 높다

array([ 1.62588263,  5.9313457 ,  4.33385186, ...,  0.47754467,
       -9.27302276,  0.90485881])

3030

In [56]:
# 해당 상관계수에 순차적으로 번호를 붙인 튜플 형태로 만들기 위해서 enumerate 함수를 이용
a2 = list(enumerate(a1))
a2

[(0, 1.6258826263308872),
 (1, 5.931345700126452),
 (2, 4.333851862970069),
 (3, 2.3859064658957183),
 (4, 1.8653806263183768),
 (5, 1.4046298707332427),
 (6, 4.960703969940273),
 (7, 9.668211340433682),
 (8, 3.5546579361657504),
 (9, 1.147042728764386),
 (10, 0.7042842607234275),
 (11, 0.6028054248027678),
 (12, 0.7615770091676731),
 (13, 0.5430034540340465),
 (14, -5.776360160743486),
 (15, 5.117483504013064),
 (16, -16.877755146407875),
 (17, 1.898420228491401),
 (18, -23.953359513508627),
 (19, 0.5305659753810009),
 (20, 1.3260447662580965),
 (21, 0.984489267000936),
 (22, 1.0017194148831194),
 (23, -2.084336924543416),
 (24, -17.17033144152424),
 (25, -3.471194717689617),
 (26, 0.25687748780731195),
 (27, -4.1860252176969714),
 (28, 0.6028054248027678),
 (29, 0.35331904889579974),
 (30, 7.202868887671629),
 (31, 0.008598177271946842),
 (32, 1.3025815763479351),
 (33, 2.3271700420465016),
 (34, -5.409565076418387),
 (35, 0.3799278310600885),
 (36, 0.8071915482408338),
 (37, 0.54303

In [57]:
# sorting 작업을 하기 위해서 인덱스에 있는 값을 계수로, 계수에 있는 값을 인덱스로 변경
a3 = []

for idx, value in a2 : 
	a3.append((value, idx))
    
a3

[(1.6258826263308872, 0),
 (5.931345700126452, 1),
 (4.333851862970069, 2),
 (2.3859064658957183, 3),
 (1.8653806263183768, 4),
 (1.4046298707332427, 5),
 (4.960703969940273, 6),
 (9.668211340433682, 7),
 (3.5546579361657504, 8),
 (1.147042728764386, 9),
 (0.7042842607234275, 10),
 (0.6028054248027678, 11),
 (0.7615770091676731, 12),
 (0.5430034540340465, 13),
 (-5.776360160743486, 14),
 (5.117483504013064, 15),
 (-16.877755146407875, 16),
 (1.898420228491401, 17),
 (-23.953359513508627, 18),
 (0.5305659753810009, 19),
 (1.3260447662580965, 20),
 (0.984489267000936, 21),
 (1.0017194148831194, 22),
 (-2.084336924543416, 23),
 (-17.17033144152424, 24),
 (-3.471194717689617, 25),
 (0.25687748780731195, 26),
 (-4.1860252176969714, 27),
 (0.6028054248027678, 28),
 (0.35331904889579974, 29),
 (7.202868887671629, 30),
 (0.008598177271946842, 31),
 (1.3025815763479351, 32),
 (2.3271700420465016, 33),
 (-5.409565076418387, 34),
 (0.3799278310600885, 35),
 (0.8071915482408338, 36),
 (0.543034901

In [58]:
# 상관계수를 기준으로 내림차순 정렬
coef_pos_index = sorted(a3, reverse=True)
coef_pos_index

[(37.7529091393018, 1017),
 (26.887047222978467, 1001),
 (26.51604049440025, 2246),
 (21.613079296685765, 1030),
 (21.05925257550198, 1093),
 (18.88913451820502, 999),
 (16.953422188474615, 2376),
 (16.695609497039925, 721),
 (16.11493001586686, 1388),
 (15.367162571961792, 2613),
 (15.013308941498947, 2388),
 (14.840877126003699, 1029),
 (14.087713804319518, 2330),
 (13.069147688499323, 2647),
 (12.650446735195047, 608),
 (12.507299980202498, 2404),
 (12.418365819409063, 2247),
 (12.072419753458934, 2128),
 (11.83716342540608, 873),
 (11.366648285115959, 1006),
 (11.243437975376553, 1008),
 (11.152692131918815, 2502),
 (11.070654775933663, 961),
 (10.657225361617305, 1386),
 (10.269000408106406, 635),
 (10.065231520665032, 2125),
 (9.971587516016122, 897),
 (9.725824722464397, 2363),
 (9.668211340433682, 7),
 (9.38420652338208, 607),
 (9.324908071389912, 945),
 (9.239676181617797, 1007),
 (9.225130386912156, 428),
 (9.007354616748344, 1137),
 (8.875140191929047, 1467),
 (8.56757782086

In [ ]:
# 아까 저장했던 형태소 단어장을 단어 번호를 담을 딕셔너리로 만들기
# enumerate의 순서를 바꿔줬던 것처럼 
# 형태소 단어장에 있는 단어도 시리얼넘버를 키로 하고, 해당 형태소를 값으로 반대로 바꿔준다.

In [59]:
# 새로운 딕셔너리 생성
text_data_dict = {}

# 단어 사전에 있는 단어의 수만큼 반복한다.
for key in index_vectorizer.vocabulary_ :
	# 현재 key에 해당하는 값을 가져온다.
    value = index_vectorizer.vocabulary_[key]
    
    # 위의 딕셔너리에 담는다.
    text_data_dict[value] = key
    
text_data_dict

{2647: '친절하시고/Adjective',
 428: '깔끔하고/Adjective',
 2403: '좋았습니다/Adjective',
 2356: '조용하고/Adjective',
 233: '고기/Noun',
 721: '도/Josa',
 330: '굿/Noun',
 120: '갈비탕/Noun',
 260: '과/Josa',
 528: '냉면/Noun',
 2065: '육회/Noun',
 1419: '비빔밥/Noun',
 2082: '이/Josa',
 1013: '맛있습니다/Adjective',
 671: '대/Modifier',
 2604: '체적/Noun',
 2067: '으로/Josa',
 956: '만족하나/Adjective',
 1996: '와인/Noun',
 2077: '의/Josa',
 293: '구성/Noun',
 1476: '살짝/Noun',
 1705: '아쉬움/Noun',
 1001: '맛있고/Adjective',
 1508: '서비스/Noun',
 589: '는/Josa',
 701: '더/Noun',
 2613: '최고/Noun',
 2182: '입니다/Adjective',
 24: '가/Josa',
 2177: '입/Noun',
 1897: '에서/Josa',
 553: '녹아요/Verb',
 1935: '였습니다/Verb',
 1473: '살살/Noun',
 552: '녹는/Verb',
 2615: '최상급/Noun',
 1540: '소고기/Noun',
 901: '를/Josa',
 995: '맛/Noun',
 1316: '보고왔습니다/Verb',
 302: '구워주고/Verb',
 73: '가성/Noun',
 1408: '비/Noun',
 2528: '짱/Noun',
 2688: '콜키/Noun',
 2477: '지/Josa',
 2788: '프리/Noun',
 885: '라서/Josa',
 2070: '을/Josa',
 89: '가지/Noun',
 227: '고/Josa',
 61: '가면/Noun',
 2410: '좋은/Adj

In [68]:
# 상관계수 딕셔너리에 있는 시리얼 넘버와 일치하는 키워드를 가져오기

In [62]:
# 긍정적인 키워드 상위 20개를 출력
for value, idx in top20 : 
	print(text_data_dict[idx])

맛있어요/Adjective
맛있고/Adjective
잘/Verb
맛있었어요/Adjective
먹었습니다/Verb
맛있게/Adjective
좋고/Adjective
도/Josa
분위기/Noun
최고/Noun
좋아요/Adjective
맛있었습니다/Adjective
정말/Noun
친절하시고/Adjective
다/Adverb
좋았어요/Adjective
잘/VerbPrefix
이에요/Josa
또/Noun
맛있네요/Adjective


In [63]:
# 부정적인 키워드를 출력
for value, idx in bottom20 : 
	print(text_data_dict[idx])

추웟어/Noun
한참/Noun
체인점/Noun
ㅠㅠ/KoreanParticle
가/Josa
주문/Noun
는/Josa
분/Noun
종업원/Noun
굿/Noun
적어요/Verb
를/Josa
시끄러워요/Adjective
ㅠㅠㅠㅠ/KoreanParticle
점/Noun
을/Josa
은/Josa
기대이하였음/Verb
많이/Adverb
별로/Noun


In [128]:
input_text = '딱히 대단한 것도 없는 듯~! 기대했던 것보다 별로였음 너무 부풀려진 거 같음'
# input_text = '친절하시고 깔끔하고 좋았습니다'
#입력 텍스트에 대한 전처리 수행
input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(input_text)
input_text = [" ".join(input_text)]
# print(input_text)

# 형태소로 분리
# pos_text = get_pos(input_text)
# print(pos_text)



[CountVectorizer(tokenizer=<function <lambda> at 0x000002DAE5A5E550>)]


ValueError: Expected 2D array, got 1D array instead:
array=[CountVectorizer(tokenizer=<function <lambda> at 0x000002DAE5A5E550>)].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [69]:





# 최적 감성 분석 모델에 적용하여 감성 분석 평가
st_predict = model.predict(X=)

#예측 결과 출력
if(st_predict == 0):
    print('예측 결과: ->> 부정 감성')
else :
    print('예측 결과: ->> 긍정 감성')

ValueError: could not convert string to float: '딱히 대단한 것도 없는 듯 기대했던 것보다 별로였음 너무 부풀려진 거 같음'